<a href="https://colab.research.google.com/github/Paengpeng/229351-StatisticalLearning/blob/main/229351_LAB05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ปฏิบัติการครั้งที่ 5 กระบวนวิชา 229351 Statistical Learning for Data Science

คำชี้แจง

ให้เริ่มทำปฏิบัติการจาก colab notebook ที่กำหนดให้ จากนั้น share แล้วส่ง link มาใน mango.cmu.ac.th

ดาวน์โหลดข้อมูลรถยนต์ชนิดต่างใน link ข้างล่างนี้
https://donlapark.pages.dev/229351/data/Auto.csv
หรือรันโค้ดข้างล่าง

In [1]:
!wget https://donlapark.pages.dev/229351/data/Auto.csv

--2025-08-13 07:17:38--  https://donlapark.pages.dev/229351/data/Auto.csv
Resolving donlapark.pages.dev (donlapark.pages.dev)... 172.66.47.56, 172.66.44.200, 2606:4700:310c::ac42:2cc8, ...
Connecting to donlapark.pages.dev (donlapark.pages.dev)|172.66.47.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18079 (18K) [text/csv]
Saving to: ‘Auto.csv’

Auto.csv            100%[===================>]  17.66K  --.-KB/s    in 0s      

2025-08-13 07:17:39 (78.1 MB/s) - ‘Auto.csv’ saved [18079/18079]



In [2]:
import numpy as np
import pandas as pd
from scipy import stats

In [3]:
# อ่านไฟล์ csv ก่อนเพื่อหา missing values
auto_df = pd.read_csv('Auto.csv',na_values=["?"])
predictor_names = auto_df.columns

# ลบแถวที่มี missing values
auto_df = auto_df.dropna()
auto_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


### ตัวแปรต่าง ๆ มีดังนี้
mpg

    miles per gallon
cylinders

    Number of cylinders between 4 and 8
displacement

    Engine displacement (cu. inches)
horsepower

    Engine horsepower
weight

    Vehicle weight (lbs.)
acceleration

    Time to accelerate from 0 to 60 mph (sec.)
year

    Model year (modulo 100)
origin

    Origin of car (1. American, 2. European, 3. Japanese)
name

    Vehicle name


In [ ]:
auto_df = auto_df.drop(['name', 'origin'],axis=1)

auto_df.head()

## 1. Multivariate linear regression

เปลี่ยน $\texttt{dataframe}$ ให้เป็น numpy array ด้วย `dataframe.to_numpy()`

1. Response: $\texttt{y}$ เป็นคอลัมน์เวกเตอร์ของ `mpg`
2. Predictors: $\texttt{X}$ เป็นเมทริกซ์ของตัวแปรที่เหลือ (ใช้ `auto_df.drop`)


In [4]:
y = auto_df['mpg'].to_numpy()
X = auto_df.drop(['mpg'],axis=1).to_numpy()

คอลัมน์แรกของ $\texttt{X}$ ต้องเป็น $(1, 1, \ldots ,1 )^T$ (จำนวนเลข 1 เท่ากับจำนวนแถวของ $\texttt{X}$)  
สร้างเวกเตอร์ที่มีแต่เลข 1 และมีความยาวที่เหมาะสมด้วยคำสั่งใดคำสั่งหนึ่งต่อไปนี้  
1. $\texttt{[1]*X.shape[0]}$  
2. $\texttt{np.ones(X.shape[0])}$  

In [5]:
X.shape

(392, 8)

In [6]:
ones =  [1]*X.shape[0] #your answer here

# ใส่ ones ให้เป็นคอลัมน์แรกของ X
X = np.c_[ones,X]

In [7]:
#เช็คคำตอบ
print(X)

[[1 8 307.0 ... 70 1 'chevrolet chevelle malibu']
 [1 8 350.0 ... 70 1 'buick skylark 320']
 [1 8 318.0 ... 70 1 'plymouth satellite']
 ...
 [1 4 135.0 ... 82 1 'dodge rampage']
 [1 4 120.0 ... 82 1 'ford ranger']
 [1 4 119.0 ... 82 1 'chevy s-10']]


### Exercise 1: จงเขียนฟังก์ชันต่อไปนี้
1. `linear_model` มี input เป็นแมทริกซ์ `X` เวกเตอร์ `y` และ output เป็นเวกเตอร์ของสัมประสิทธิ์
2. `predict` มี input เป็นแมทริกซ์ `X` เวกเตอร์ของสัมประสิทธิ์ `beta` และ output เป็นเวกเตอร์ของค่าทำนาย `y_pred`

1. Matrix multiplication $AB = \texttt{A@B}$
2. Transpose: $X^T = \texttt{X.T}$
3. Inverse: $X^{-1} = \texttt{np.linalg.inv(X)}$

$$ \hat{\beta} = (X^TX)^{-1}X^Ty $$

In [23]:
# Let's start by implementing linear regression from scratch
# using the formula provided in the class

y = auto_df['mpg']
X = auto_df.drop(columns=['mpg', 'name'])

X.insert(0, 'intercept', 1)

X = np.array(X, dtype=float)
y = np.array(y, dtype=float)

def linear_model(X, y):
    """X: เมทริกซ์ของตัวแปรต้น"""
    """y: เวกเตอร์ของตัวแปรตาม"""
    """Return: เวกเตอร์ของ parameter beta จากการสร้าง linear regressions model
    ด้วย Ordinary Least Squares (OLS)"""
    # TODO: your code here

    X = np.array(X, dtype=float)
    y = np.array(y, dtype=float)

    XT = X.T
    beta = np.linalg.inv(XT @ X) @ (XT @ y)

    return beta


def predict(X, beta):
    """beta: array ของสัมประสิทธิ์"""
    """X: เมทริกซ์ของตัวแปรต้น"""
    """Return: ค่าทำนาย y_pred = X*beta """
    # TODO: your code here
    y_pred = X @ beta

    return y_pred

beta = linear_model(X, y)
y_pred = predict(X, beta)

print("Coefficients are:", beta)
print("Predictions are:", y_pred)

Coefficients are: [-1.72184346e+01 -4.93376319e-01  1.98956437e-02 -1.69511442e-02
 -6.47404340e-03  8.05758383e-02  7.50772678e-01  1.42614050e+00]
Predictions are: [15.00095865 13.99929917 15.24044696 15.06190592 14.96717762 10.69562338
 10.6553509  10.69318951 10.21140958 13.11319398 15.29186157 14.14690266
 14.64696189 18.88015369 24.13062472 19.03847473 19.40643664 20.88068005
 25.45738302 27.13648841 21.03963303 22.25408365 22.7074409  23.25050717
 20.3005362   7.58306997  8.41074023  8.30037285  6.44696729 26.20815569
 23.39053912 25.73308381 21.46783861 16.23364804 17.52793963 17.90615253
 17.43525382 11.44975337 10.58385513 12.14999889 11.87950268  6.98211816
  8.88321853  6.24768971 19.73241559 23.0454129  17.79193175 18.88084577
 23.26421626 25.13116053 25.49443983 25.22718096 28.75457004 29.66193221
 27.53555535 25.15803406 26.28101808 24.47954013 26.03152794 23.5313529
 24.17758427 11.77971323 11.88636515 12.41807558 13.02233473 14.97586698
 10.21781246 10.59400653 10.8070

#### จงหา $n$ (จำนวนแถวของ $\texttt{X}$) และ $p$ (จำนวนตัวแปร) ด้วย method $\texttt{X.shape[...]}$

In [24]:
n = X.shape[0]

p = X.shape[1]-1

print('Number of observations:', n)
print('Number of variables:', p)

Number of observations: 392
Number of variables: 7


## 2. F-statistic

### Exercise 2: เขียนสมมติฐานหลัก ($H_0$) และสมมติฐานแย้ง ($H_a$) ที่ใช้ F-statistic ในการทดสอบ

สมมติฐานหลัก (Null Hypothesis, H₀)
ทุกสัมประสิทธิ์ของตัวแปรต้น ไม่ต่างจากศูนย์ (ไม่มีตัวแปรใดมีผลต่อ y)
H₀ : β1=β2==βp=0

สมมติฐานแย้ง (Alternative Hypothesis, Hₐ)
อย่างน้อยหนึ่งตัวแปรต้นมีผลต่อ y (สัมประสิทธิ์ ≠ 0)
Hₐ : อย่างน้อยหนึ่ง βⱼ ≠ 0, j=1,...,p

### Exercise 3: จากเวกเตอร์ของ outcomes `y` คำนวณ Total Sum of Squares (TSS)

$$ \text{TSS} = \sum_{i=1}^n (y_i-\bar{y})^2 $$

$$ \text{RSS} = \sum_{i=1}^n (y_i-\hat{y})^2 $$
โดยใช้คำสั่งต่อไปนี้

$\texttt{np.sum}([y_1,y_2,...,y_n])=y_1+y_2+...+y_n$

$\texttt{np.square}([y_1,y_2,...,y_n])=[y_1^2,y^2_2,...,y^2_n]$

In [27]:
# Calculate Total Sum of Squares
y_mean = y.mean()

# TODO: your code here
TSS = np.sum(np.square(y - y_mean))

RSS = np.sum(np.square(y-y_pred))
print("Total Sum of Squares (TSS):", TSS)
print("Residual Sum of Squares (RSS):", RSS)

Total Sum of Squares (TSS): 23818.99346938776
Residual Sum of Squares (RSS): 4252.212530440178


คำนวณ $F$-statistic ด้วย
$$F_{p,n-p-1} = \frac{(\text{TSS}-\text{RSS})/p}{\text{RSS}/(n-p-1)}$$
หา $n$ และจำนวนของตัวแปร $p$ ด้วย $\texttt{X.shape}[\cdot]$

### Exercise 4: คำนวณค่า F

In [28]:
F = ((TSS - RSS)/p) / (RSS / (n - p - 1))

print(F)

252.42804529131897


In [29]:
# Compute p_value of the F-statistic

p_value = stats.f.sf(F, p, n-p-1)

print(p_value)

2.0371059307552074e-139


### Exercise 5: จากการทดสอบสมมติฐานที่นัยสำคัญ 0.05 เราได้ข้อสรุปอย่างไรจากค่า p-value ที่ได้ข้างบน

### คำตอบของ Exercise 5:




## 3. คำนวณ $t$-statistic

#### ขั้นตอนในการคำนวณ $t$-statistic

\begin{align*}
\text{RSE} &= \sqrt{\frac{\text{RSS}}{n-p-1}} \\
\text{SE}(\hat{\beta}) &=\text{RSE}\cdot\sqrt{\text{diag}\left( (X^TX)^{-1}\right)} \\
t &= \frac{\hat{\beta}}{\text{SE}(\hat{\beta})}
\end{align*}

In [33]:
n = X.shape[0]
p = X.shape[1] - 1

RSE = np.sqrt(RSS / (n - p - 1))

XTX_inv = np.linalg.inv(X.T @ X)
SE_beta = RSE * np.sqrt(np.diag(XTX_inv))

t = beta / SE_beta

p_values = t_to_p_values(t)
print(p_value)

2.0371059307552074e-139


### Exercise 6: จงใช้ฟังก์ชัน `t_to_p_values` ในการหาว่ามีตัวแปรใดบ้างที่มีค่า p-value น้อยกว่า 0.05 **แล้วระบุตัวแปรทั้งหมดใน text block ข้างล่าง**  

#### ใช้คำสั่ง `A.diagonal()` เพื่อดึงค่าในแนวทแยงของ `A`

In [47]:
# TODO: your code here
predictor_names = X_df.columns


X = np.array(X_df, dtype=float)


RSE = np.sqrt(RSS / (n - p - 1))


XTX_inv = np.linalg.inv(X.T @ X)
SE = RSE * np.sqrt(XTX_inv.diagonal())


t = beta / SE


p_values = t_to_p_values(t)


significant_vars = predictor_names[p_values < 0.05]


print("Variables with p-value < 0.05:")
print(significant_vars)

Variables with p-value < 0.05:
Index(['intercept', 'displacement', 'weight', 'year', 'origin'], dtype='object')


intercept

displacement

weight

year

origin

list ชื่อตัวแปร

In [35]:
print("List of predictors:", predictor_names[1:-2])

List of predictors: Index(['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration',
       'year'],
      dtype='object')


### คำตอบของ Exercise 6:





## 4. Variable selection

#### statsmodels library สามารถคำนวณค่าต่างๆ เหล่านี้ได้ ผลที่แสดงจะคล้ายกับใน R  
#### มีสองวิธึในการทำ linear regression: $\texttt{statsmodels.api.sm.OLS}$
#### และ $\texttt{statsmodels.formula.api.smf.ols}$

In [50]:
import statsmodels.api as sm
X = auto_df.select_dtypes(include=[np.number]).drop(['mpg'], axis=1)
X = sm.add_constant(X)
y = auto_df['mpg']
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.821
Model:                            OLS   Adj. R-squared:                  0.818
Method:                 Least Squares   F-statistic:                     252.4
Date:                Wed, 13 Aug 2025   Prob (F-statistic):          2.04e-139
Time:                        08:53:12   Log-Likelihood:                -1023.5
No. Observations:                 392   AIC:                             2063.
Df Residuals:                     384   BIC:                             2095.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -17.2184      4.644     -3.707   

In [51]:
import statsmodels.formula.api as smf

model2 = smf.ols('mpg ~ cylinders + displacement + horsepower + weight + acceleration + year', auto_df)
model2 = model2.fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.809
Model:                            OLS   Adj. R-squared:                  0.806
Method:                 Least Squares   F-statistic:                     272.2
Date:                Wed, 13 Aug 2025   Prob (F-statistic):          3.79e-135
Time:                        08:53:39   Log-Likelihood:                -1036.5
No. Observations:                 392   AIC:                             2087.
Df Residuals:                     385   BIC:                             2115.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      -14.5353      4.764     -3.051   

### Exercise 7: ใช้ Backward selection ในการเลือกตัวแปร โดยที่เราจะหยุดเมื่อ p-value สของตัวแปรทุกตัว < 0.05 แล้วเขียนสมการของโมเดลที่ได้

In [53]:
# Block for running your own code
predictors = ['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']  # ปรับตาม dataset
formula = 'mpg ~ ' + ' + '.join(predictors)

In [54]:
# If you want to write with for-loop
while True:
    model = smf.ols(formula, data=auto_df).fit()
    p_values = model.pvalues.drop('Intercept')
    max_p = p_values.max()

    if max_p < 0.05:
        break
    else:
        worst_feature = p_values.idxmax()
        predictors.remove(worst_feature)
        formula = 'mpg ~ ' + ' + '.join(predictors)


final_model = smf.ols(formula, data=auto_df).fit()
print(final_model.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.820
Model:                            OLS   Adj. R-squared:                  0.818
Method:                 Least Squares   F-statistic:                     351.7
Date:                Wed, 13 Aug 2025   Prob (F-statistic):          2.73e-141
Time:                        08:57:15   Log-Likelihood:                -1025.1
No. Observations:                 392   AIC:                             2062.
Df Residuals:                     386   BIC:                             2086.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      -16.6939      4.120     -4.051   

### คำตอบของ Exercise 7:

$$
y =
$$

ŷ = β0 + β1 ⋅ displacement + β2 ⋅ weight + β3 ⋅ year + β4 ⋅ origin